<a href="https://colab.research.google.com/github/John-the-ripp/Event/blob/master/title_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.3 MB/s eta 0:00:00


In [ ]:
import json
def preprocess_intents_json(intents_file):
    with open(intents_file, "r") as f:
        data = json.load(f)

    preprocessed_data = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            preprocessed_data.append(f"User: {pattern}\n")
            for response in intent["responses"]:
                preprocessed_data.append(f"Assistant: {response}\n")

    return "".join(preprocessed_data)

def save_preprocessed_data(preprocessed_data, output_file):
    with open(output_file, "w") as f:
        f.write(preprocessed_data)

intents_file = "/content/drive/MyDrive/intents.json"
output_file = "/content/drive/MyDrive//mental_health_data.txt"

preprocessed_data = preprocess_intents_json(intents_file)
save_preprocessed_data(preprocessed_data, output_file)

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

In [4]:
def fine_tune_gpt2(model_name, train_file, output_dir):
    # Load GPT-2 model and tokenizer
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load training dataset
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_file,
        block_size=128)
    # Create data collator for language modeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False)
    # Set training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=50,
        per_device_train_batch_size=4,
        save_steps=20_000,
        save_total_limit=10,
    )
    # Train the model
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )
    trainer.train()
    # Save the fine-tuned model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

In [5]:
# Fine-tune the model
fine_tune_gpt2("gpt2", "/content/title anas.txt", "output_dir_gpt2_fineTuned_model_titles")

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.082100
1000,1.769600
1500,1.564200
2000,1.349400
2500,1.187500
3000,1.043600
3500,0.897100
4000,0.791700
4500,0.696100
5000,0.605900


Step,Training Loss
500,2.082100
1000,1.769600
1500,1.564200
2000,1.349400
2500,1.187500
3000,1.043600
3500,0.897100
4000,0.791700
4500,0.696100
5000,0.605900


In [6]:
tokenizer = AutoTokenizer.from_pretrained("output_dir_gpt2_fineTuned_model_titles")
model_config = GPT2Config.from_pretrained("output_dir_gpt2_fineTuned_model_titles/config.json")
model = AutoModelForCausalLM.from_pretrained("output_dir_gpt2_fineTuned_model_titles/pytorch_model.bin", config=model_config)


In [9]:
# Create a text generation pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generate text using the fine-tuned model
prompt ="Q: Can you give me a title about 'big data'"
generated_text = text_generator(prompt, max_length=35, num_return_sequences=1)

# Print the generated text
print(generated_text[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Can you give me a title about 'big data'
A: Title:Iterative Hierarchy and Ranking Process (IHRP): A Novel Effective Hierarchy


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
